In [83]:
import nltk
import numpy as np
import pandas as pd
import re
from numpy import triu

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /Users/mac/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/mac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/mac/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [84]:
df = pd.read_csv('SMSSpamCollection.csv', sep='\t', names=['label', 'message'])

df.head()


,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [85]:
corpus = []

In [86]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()


In [87]:
for i in range (0, len(df)):
    result = re.sub('[^a-zA-Z]', ' ', df['message'][i])
    result = result.lower()
    result = result.split()
    result = [lemmatizer.lemmatize(word) for word in result if not word in set(stopwords.words('english'))]
    result = ' '.join(result)
    corpus.append(result)


In [ ]:
for i,j,k in zip(df['label'], df['message'], corpus):
    print(i, j, k)

In [88]:
corpus

['go jurong point crazy available bugis n great world la e buffet cine got amore wat',
 'ok lar joking wif u oni',
 'free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate c apply',
 'u dun say early hor u c already say',
 'nah think go usf life around though',
 'freemsg hey darling week word back like fun still tb ok xxx std chgs send rcv',
 'even brother like speak treat like aid patent',
 'per request melle melle oru minnaminunginte nurungu vettam set callertune caller press copy friend callertune',
 'winner valued network customer selected receivea prize reward claim call claim code kl valid hour',
 'mobile month u r entitled update latest colour mobile camera free call mobile update co free',
 'gonna home soon want talk stuff anymore tonight k cried enough today',
 'six chance win cash pound txt csh send cost p day day tsandcs apply reply hl info',
 'urgent week free membership prize jackpot txt word claim c www dbuk net lccltd pobox ldnw rw'

In [99]:
from nltk.tokenize import sent_tokenize
from gensim.utils import simple_preprocess


In [90]:
words = []

for sentence in corpus:
    sent_tokenize_list = sent_tokenize(sentence)
    for s in sent_tokenize_list:
        words.append(simple_preprocess(s))

In [91]:
words

[['go',
  'jurong',
  'point',
  'crazy',
  'available',
  'bugis',
  'great',
  'world',
  'la',
  'buffet',
  'cine',
  'got',
  'amore',
  'wat'],
 ['ok', 'lar', 'joking', 'wif', 'oni'],
 ['free',
  'entry',
  'wkly',
  'comp',
  'win',
  'fa',
  'cup',
  'final',
  'tkts',
  'st',
  'may',
  'text',
  'fa',
  'receive',
  'entry',
  'question',
  'std',
  'txt',
  'rate',
  'apply'],
 ['dun', 'say', 'early', 'hor', 'already', 'say'],
 ['nah', 'think', 'go', 'usf', 'life', 'around', 'though'],
 ['freemsg',
  'hey',
  'darling',
  'week',
  'word',
  'back',
  'like',
  'fun',
  'still',
  'tb',
  'ok',
  'xxx',
  'std',
  'chgs',
  'send',
  'rcv'],
 ['even', 'brother', 'like', 'speak', 'treat', 'like', 'aid', 'patent'],
 ['per',
  'request',
  'melle',
  'melle',
  'oru',
  'minnaminunginte',
  'nurungu',
  'vettam',
  'set',
  'callertune',
  'caller',
  'press',
  'copy',
  'friend',
  'callertune'],
 ['winner',
  'valued',
  'network',
  'customer',
  'selected',
  'receivea',
 

In [112]:
from gensim.models import Word2Vec

model = Word2Vec(words)


ImportError: cannot import name 'triu' from 'scipy.linalg' (/Users/mac/Desktop/MLOps/venv/lib/python3.12/site-packages/scipy/linalg/__init__.py)

In [111]:
print(model['free'])

model.wv.most_similar('free')

model.wv.most_similar('call')

model.wv.most_similar('win')

model.wv.similarity('free', 'call')

model.wv.cosine_similarities(model['free'], [model['call'], model['win']])

NameError: name 'model' is not defined

In [110]:
def avg_word2vec(doc):
    return np.mean([model[word] for word in doc if word in model] or [np.zeros(100)], axis=0)

In [108]:
!pip install tqdm

In [109]:
X = []

from tqdm import tqdm

for doc in tqdm(words):
    X.append(avg_word2vec(doc))

  0%|          | 0/5564 [00:00<?, ?it/s]


NameError: name 'model' is not defined

In [114]:
X_new = np.array(X)
y = df[list(map(lambda x: len(x)>0,corpus))]
y = df['label'].map({'ham': 0, 'spam': 1})

In [ ]:
new_df = pd.DataFrame()

for i in range(0, len(X_new)):
    new_df = new_df.append(pd.DataFrame(X_new[i].reshape(1, -1)),ignore_index=True)

In [ ]:
new_df.isnull().sum()

In [ ]:
new_df["Output"] = y

new_df.to_csv('word2vec.csv', index=False)

new_df.dropna(inplace=True)

new_df.isnull().sum()

In [ ]:
X = new_df.drop('Output', axis=1)
y = df['Output'].map({'ham': 0, 'spam': 1})

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.ensemble import RandomForestClassifier

rfs = RandomForestClassifier(n_estimators=100, random_state=42)

rfs.fit(X_train, y_train)

y_pred = rfs.predict(X_test)

from sklearn.metrics import accuracy_score, confusion_matrix

accuracy_score(y_test, y_pred)

confusion_matrix(y_test, y_pred)

from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

